In [1]:
import pyspark
import time


In [2]:
import csv
mydict={}
with open('la.lexicon.csv', mode='r') as infile:
    reader = csv.reader(infile)
    
    for rows in reader:
        rows[0]=rows[0].replace('j','i')
        rows[0]=rows[0].replace('v','u').lower()
        rows[2]=rows[2].lower()
        if mydict.has_key(rows[0]) and rows[2] not in mydict.get(rows[0]):
            val = mydict[rows[0]]
            mydict[rows[0]] = val.append(rows[2])
            mydict[rows[0]] = val
        else:
            val=[]
            val.append(rows[0])
            val.append(rows[2])
            mydict[rows[0]] = val
           # print(mydict)
            

In [3]:
mydict.get('aban')

['aban', 'aba', 'abas']

In [4]:
from pyspark import SparkContext
sc =SparkContext()

In [5]:

lines = []
lines = sc.textFile("data.txt")


In [8]:
from operator import add
return_dict = {}
# for line in file_handle:
# 	line_list = line.split()
# 	try:
# 	        return_dict[line_list[0]].append(line_list)
# 	except KeyError:
# return_dict[line_list[0]] = [line_list]
# def returnLemma(l):
    
#     if(mydict.has_key(l)):
        
#         return mydict.get(l).split(',')
#     else:
        
#         return l
def returnLemma(l):
    
    try:
        return mydict.get(l).split(',')
    except:
        return l
    
start=time.time()

location_line = lines.map(lambda line: line.split('\t'))

#linesSplit = sc.parallelize()
reduced_op=location_line.map(lambda y: (y[0],y[-1].split(' ')))\
    .flatMap(lambda (x,z) : [((returnLemma(z[i]),returnLemma(z[j])),x) for i in range(0,len(z)-1) for j in range(i+1,len(z))])\
    .flatMap(lambda (k,r) :[((lem1,lem2),r) for lem1 in k[0] for lem2 in k[1]])\
    .reduceByKey(add).collect()
#     .reduceByKey(lambda a,b: a + "," + b).collect()
# a.collectAsMap()`
print(time.time()-start)
##reduced_op

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/hadoop/spark/python/pyspark/rdd.py", line 2407, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/hadoop/spark/python/pyspark/rdd.py", line 2407, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/hadoop/spark/python/pyspark/rdd.py", line 346, in func
    return f(iterator)
  File "/home/ha

In [ ]:
reduced_op

In [21]:

lines = []
lines = sc.textFile("data.txt")

In [33]:
from operator import add
import itertools
from itertools import product
# def returnLemma(l):
#     temp=[]
#     if(mydict.has_key(l)):
#         #temp.append(mydict.get(l))
#         return mydict.get(l)
#     else:
#         temp=[]
#         temp.append(l)
#         return temp
def returnLemma1(l):
    
    try:
        print mydict.get(l)
        return mydict.get(l)
    except:
        temp=[]
        temp.append(l)
#         print temp
        return temp
    #print(type(temp))
    #return temp

In [34]:

start=time.time()

location_line = lines.map(lambda line: line.split('\t'))

reduced_op1=location_line.map(lambda y: (y[0],y[-1].split(' ')))\
    .flatMap(lambda (x,z) :[((crosslema),x) for i in itertools.combinations(z,2)  for crosslema in product(returnLemma1(i[0]),returnLemma1(i[1])) ] ).collect()

# reduced_op1=location_line.map(lambda y: (y[0],y[-1].split(' ')))\
#      .flatMap(lambda (x,z) :[((crosslema),x) for i in itertools.combinations(z,2) for crosslema in product(returnLemma(i[0]),returnLemma(i[1]))] )\
#     .collect()


# #     .flatMap(lambda (x,z) :[((i),z) for i in itertools.combinations(x,2)])\
#     .reduceByKey(add).collect()
#     .flatMap(lambda (x,z) : [((['|'.join(i) for i in itertools.combinations_with_replacement(z,2)]),x)])\
#     .flatMap(lambda (k,r) :[((returnLemma(i)),r) for i in k]).collect()
#     .flatMap(lambda (k,r) :[((k),r) ]).collect()
#     .flatMap(lambda (k,r) :[((lem1,lem2),r) for lem1 in returnLemma(k.split('|')[0]) for lem2 in returnLemma(k.split('|')[1])]).collect()
#     .flatMap(lambda (k,r) :[((k.split('|')[0],k.split('|')[1]),r) ]).collect()
#     .flatMap(lambda (k,r) :[((lem1,lem2),r) for lem1 in k[0] for lem2 in k[1]]).collect()
#     .reduceByKey(add).collect()
#     .reduceByKey(lambda a,b: a + "," + b).collect()
# a.collectAsMap()`
print(time.time()-start)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 13, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-34-c9e3e1c1f041>", line 6, in <lambda>
TypeError: 'NoneType' object is not iterable

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.pyth

In [ ]:
reduced_op1

In [ ]:
reduced_op1


In [ ]:
returnLemma('abactum')

In [ ]:
alphabets=('aban,aba,abas,aban,aba,abas')
import itertools
from itertools import product
keywords = [','.join(i) for i in itertools.combinations_with_replacement(alphabets,2)]

In [ ]:
keywords

In [ ]:
for i in itertools.combinations_with_replacement(alphabets,2):
    print(i[1])